In [2]:
import sqlite3
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np


In [4]:
conn = sqlite3.connect("../../places.sqlite")

full_data = pd.read_sql_query(
    """

SELECT moz_historyvisits.id,
    moz_places.url as url,
    moz_historyvisits.'visit_date' as 'visit_date',
    moz_historyvisits.from_visit as from_visit,
    t2.url as from_url
FROM moz_historyvisits
    JOIN moz_places ON moz_places.id = moz_historyvisits.place_id
    JOIN (
        SELECT moz_historyvisits.id,
            moz_places.url as url
        FROM moz_historyvisits
            JOIN moz_places ON moz_places.id = moz_historyvisits.place_id
    ) AS t2 ON t2.id = moz_historyvisits.from_visit;
    
""",
    conn,
)

# conn = sqlite3.connect("../../history.sqlite")

# full_data = pd.read_sql_query(
#     """

# SELECT visits.id,
#     urls.url as url,
#     visits.'visit_time' as 'visit_date',
#     visits.from_visit as from_visit,
#     t2.url as from_url
# FROM visits
#     JOIN urls ON urls.id = visits.url
#     JOIN (
#         SELECT visits.id,
#             urls.url as url
#         FROM visits
#             JOIN urls ON urls.id = visits.url
#     ) AS t2 ON t2.id = visits.from_visit;

# """,
#     conn,
# )


# SELECT moz_historyvisits.id,
#     moz_places.url as url,
#     moz_historyvisits.'visit_date' as 'visit_date',
#     moz_historyvisits.from_visit as from_visit,
#     NULL AS from_url
# FROM moz_historyvisits
#     JOIN moz_places ON moz_places.id = moz_historyvisits.place_id
# WHERE moz_historyvisits.from_visit NOT IN (
#         SELECT id
#         FROM moz_historyvisits
#     )
# UNION


In [5]:
full_data.head()


,id,url,visit_date,from_visit,from_url
0,70358,https://leetcode.com/submissions/,13333532484545657,70357,https://leetcode.com/Manan_144/
1,70359,https://leetcode.com/submissions/#/,13333532485476379,70358,https://leetcode.com/submissions/
2,70360,https://leetcode.com/submissions/#/1,13333532485478790,70359,https://leetcode.com/submissions/#/
3,70361,https://leetcode.com/submissions/detail/991140...,13333532490722880,70358,https://leetcode.com/submissions/
4,70362,https://leetcode.com/problems/minimum-falling-...,13333532526031718,70361,https://leetcode.com/submissions/detail/991140...


In [6]:
import csv
import re

full_data["from_host"] = full_data["from_url"].map(
    lambda x: re.match(r"https?:\/\/(([^/]+\.)+[^/]+)", x).group(1)
    if x is not None and re.match(r"https?:\/\/(([^/]+\.)+[^/]+)", x) is not None
    else pd.NA
)

full_data["to_host"] = full_data["url"].map(
    lambda x: re.match(r"https?:\/\/(([^/]+\.)+[^/]+)", x).group(1)
    if re.match(r"https?:\/\/(([^/]+\.)+[^/]+)", x) is not None
    else pd.NA
)

full_data["to_host"].isna().sum()


25

In [7]:
from datetime import datetime

full_data["visit_date_obj"] = full_data["visit_date"].map(
    lambda x: datetime.fromtimestamp(x / 1000000)
)


In [8]:
full_data.drop("url", axis=1, inplace=True)
full_data.drop("from_url", axis=1, inplace=True)
full_data["visit_count"] = 1

full_data.dropna(axis=0, how="any", inplace=True)
full_data.reset_index(inplace=True, drop=True)

full_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11217 entries, 0 to 11216
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              11217 non-null  int64 
 1   visit_date      11217 non-null  int64 
 2   from_visit      11217 non-null  int64 
 3   from_host       11217 non-null  object
 4   to_host         11217 non-null  object
 5   visit_date_obj  11217 non-null  object
 6   visit_count     11217 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 613.6+ KB


In [9]:
full_data.head()


,id,visit_date,from_visit,from_host,to_host,visit_date_obj,visit_count
0,70358,13333532484545657,70357,leetcode.com,leetcode.com,2392-07-10 07:01:24.545656,1
1,70359,13333532485476379,70358,leetcode.com,leetcode.com,2392-07-10 07:01:25.476379,1
2,70360,13333532485478790,70359,leetcode.com,leetcode.com,2392-07-10 07:01:25.478790,1
3,70361,13333532490722880,70358,leetcode.com,leetcode.com,2392-07-10 07:01:30.722879,1
4,70362,13333532526031718,70361,leetcode.com,leetcode.com,2392-07-10 07:02:06.031717,1


In [10]:
unique_hosts = full_data["from_host"].unique()
full_data["visits"] = 1

final_data = full_data.groupby(["to_host", "from_host"]).agg({"visits": "sum"})

final_data.reset_index(inplace=True)

final_data.head()


,to_host,from_host,visits
0,10.0.0.1:4100,10.0.0.1:4100,4
1,101desires.com,101desires.com,34
2,101desires.com,dotmovies.icu,2
3,101desires.com,www.google.com,2
4,1x-bet.in,refpamjeql.top,1


In [15]:
def top_n(host: str, n: int):
    x = final_data[final_data["from_host"] == host]
    x = x.sort_values("visits", ascending=False)

    tot_visits = x["visits"].sum()
    x["probability"] = x["visits"].map(lambda x: x / tot_visits)

    res = x.head(n)[["to_host", "probability"]]

    print(f"From: {host}\n")
    print(res)
    print("")


top_n("www.google.com", 5)
top_n("www.youtube.com", 5)
top_n("github.com", 15)


From: www.google.com

                   to_host  probability
688         www.google.com     0.311747
676  www.geeksforgeeks.org     0.084337
718   www.interviewbit.com     0.027108
173        click.pstmrk.it     0.027108
510      stackoverflow.com     0.024096

From: www.youtube.com

              to_host  probability
800   www.youtube.com     0.973441
330        github.com     0.003634
522  takeuforward.org     0.002796
321   gist.github.com     0.001957
267  drive.google.com     0.001677

From: github.com

                                               to_host  probability
325                                         github.com     0.952688
60                                     api.netlify.com     0.015054
296                       firstcontributions.github.io     0.006452
485                                    reactrouter.com     0.006452
320                                    gist.github.com     0.004301
116                 blog-app-mananmehta1404.vercel.app     0.002151
341      